<a href="https://colab.research.google.com/github/tayfununal/nku_lesson/blob/main/CIFAR10_with_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization, GlobalAveragePooling2D, add
from tensorflow.keras import Sequential, Model, utils, optimizers, losses
from tensorflow.keras.datasets import cifar10

In [ ]:
# Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Normalization
x_train, x_test = x_train.astype('float32') / 255., x_test.astype('float32') / 255.

In [ ]:
#One - Hot Encoding
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

In [ ]:
#Model

inputs = Input(shape=(32, 32, 3), name="img")
x = Conv2D(64, 3, activation="relu")(inputs)
x = Conv2D(64, 3, activation="relu")(x)
x = BatchNormalization()(x)
block_1_output = MaxPool2D((3,3))(x)

x = Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = Conv2D(64, 3, activation="relu", padding="same")(x)
x = BatchNormalization()(x)
block_2_output = add([x, block_1_output])

x = Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = Conv2D(64, 3, activation="relu", padding="same")(x)
x = BatchNormalization()(x)
block_3_output = add([x, block_2_output])

x = Conv2D(64, 3, activation="relu", padding="same")(block_3_output)
x = Conv2D(64, 3, activation="relu", padding="same")(x)
x = BatchNormalization()(x)
block_4_output = add([x, block_3_output])

x = Conv2D(64, 3, activation="relu", padding='same')(block_4_output)
x = Conv2D(64, 3, activation="relu", padding='same')(x)
x = BatchNormalization()(x)
block_5_output = add([x, block_4_output])

x = Conv2D(64, 3, activation="relu", padding="same")(block_5_output)
x = Conv2D(64, 3, activation="relu", padding="same")(x)
x = BatchNormalization()(x)
block_6_output = add([x, block_5_output])

x = Conv2D(64, 3, activation="relu")(block_6_output)
x = Conv2D(64, 3, activation="relu")(x)
x = BatchNormalization()(x)

x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.2)(x)

x = Dense(512, activation="relu")(x)
x = Dropout(0.2)(x)

x = Dense(256, activation="relu")(x)
x = Dropout(0.2)(x)
outputs = Dense(10)(x)

model = Model(inputs, outputs, name="ResNet")
model.summary()

In [ ]:
model.compile(
    optimizer= optimizers.Adam(1e-3),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)

In [ ]:
# Model training and creating validation set
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

In [ ]:
score = model.evaluate(x_test, y_test, batch_size = 128)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

In [ ]:
# Ploting loss and accuracy values
print(history.history.keys())
 
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()